In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df =  pd.read_csv('big_news_train.csv')
test_df =  pd.read_csv('big_news_test.csv')
test_label =  pd.read_csv('big_news_test_label.csv')

In [3]:
len(train_df)

20800

In [4]:
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
train_df = train_df[['text', 'label']]
train_df.head(3)

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1


In [6]:
train_df.isna().sum()

text     39
label     0
dtype: int64

In [7]:
train_df.dropna(inplace=True)
train_df.isna().sum()

text     0
label    0
dtype: int64

In [8]:
train_data = train_df['text']
train_label = train_df['label']

In [9]:
test_df.head(3)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...


In [10]:
test_label.head(3)

,id,label
0,20800,0
1,20801,1
2,20802,0


In [11]:
len(test_df), len(test_label)

(5200, 5200)

In [12]:
test_df['text'].isna().sum()

7

In [13]:
test_df['label'] = test_label['label']
test_df.head(3)

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0


In [14]:
new_test_df = test_df[['text', 'label']]
new_test_df.head(3)

,text,label
0,"PALO ALTO, Calif. — After years of scorning...",0
1,Russian warships ready to strike terrorists ne...,1
2,Videos #NoDAPL: Native American Leaders Vow to...,0


In [15]:
new_test_df.dropna(inplace=True)

C:\Users\Chiebuka\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
new_test_df.isna().sum()

text     0
label    0
dtype: int64

In [17]:
len(new_test_df)

5193

In [18]:
test_data = new_test_df['text']
test_label =new_test_df['label']

In [19]:
len(train_data), len(train_label), len(test_data), len(test_label)

(20761, 20761, 5193, 5193)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [21]:
def find_best_model(x, y):
    pipe_1 = clf = Pipeline([
                  ('vectorizer', CountVectorizer()),
                   ('svc', SVC(gamma='auto', probability= True))
                     ])
    pipe_2 = Pipeline([
                  ('vectorizer', CountVectorizer()),
                   ('rf', RandomForestClassifier())
                     ])
    
    pipe_3 = Pipeline([
                  ('vectorizer', CountVectorizer()),
                   ('nb', MultinomialNB())
                     ])
    config = {
            'support vector machine' : {
        'model' : pipe_1,
        'params': {
            'svc__C': [1, 10, 100, 1000],
            'svc__kernel': ['rbf', 'linear']
                }
            },
             
        'random forest classifier' : {
         'model' : pipe_2,
         'params': {
             'randomforestclassifier__criterion' : ['gini', 'entropy'],
             'randomforestclassifier__n_estimators': [1,5,10],
              'randomforestclassifier__warm_start' : [True, False]    
        }
     },
        'multinomial nb' : {
         'model' : pipe_3,
         'params': {
                
        }
     },

        }
    scores = []
    best_estimator = {}
    for model_name, model_params in config.items():
        clf = GridSearchCV(model_params['model'], model_params['params'], cv = 5, return_train_score= False)
        clf.fit(x,y)
        scores.append({
                'model' : model_name,
                'best_score' : clf.best_score_,
                'best_params' : clf.best_params_
            })
        best_estimators[alg] = clf.best_estimator_
    return best_estimator, pd.DataFrame(scores)

In [ ]:
best_estimator, scores_df = find_best_model(train_data, train_label)

In [ ]:
scores_df

In [ ]:
best_model = best_estimator['']
best_model

In [ ]:
best_model.score(test_data, test_label)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_label, best_model.predict(test_data))

In [ ]:
import seaborn as sn
plt.figure(figsize =(12,5))
sn.heatmap(cm, annot=True)
plt.ylabel('True')
plt.xlabel('predicted')

In [ ]:
data = pd.read_csv('small_news_test.csv')
data.head(3)

In [ ]:
data = data[['title', 'text', 'label']]
data.head(3)

In [ ]:
data.isna().sum()

In [ ]:
len(data)

In [ ]:
data['label_n'] = data['label'].apply(lambda x: 1 if x == 'REAL' else 0)
data.head(5)

In [ ]:
x = data['text']
y = data['label_n']

In [ ]:
x.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [ ]:
best_estimator, scores_df2 = find_best_score(x_train, y_train)

In [ ]:
score_df2

In [ ]:
best_model = best_estimator[]
best_model

In [ ]:
best_model.score(x_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, best_model.predict(x_test))

In [ ]:
import seaborn as sn
plt.figure(figsize =(12,5))
sn.heatmap(cm, annot=True)
plt.ylabel('True')
plt.xlabel('predicted')